In [1]:
library(tidyverse)

Warning message:
“replacing previous import by ‘tidyr::%>%’ when loading ‘broom’”Warning message:
“replacing previous import by ‘tidyr::gather’ when loading ‘broom’”Warning message:
“replacing previous import by ‘tidyr::spread’ when loading ‘broom’”── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [34]:
data_df <- read_tsv("expression_exploration/ExpressionExploration//data/bull_data.tsv")
design_df <- read_tsv("expression_exploration/ExpressionExploration//data/bull_design.tsv")


Parsed with column specification:
cols(
  .default = col_double(),
  `Protein IDs` = col_character(),
  `Majority protein IDs` = col_character()
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  sample = col_character(),
  samp_nbr = col_integer(),
  bull = col_character(),
  all_high_fert = col_character(),
  kviga_high_fert = col_character(),
  all_high_fert_val = col_double(),
  kviga_high_fert_val = col_double(),
  in_mid_range = col_character()
)


In [35]:
colnames(data_df)[which(!colnames(data_df) %in% design_df$sample)]

[1] "Protein IDs"          "Majority protein IDs"

In [36]:
glimpse(design_df)

Observations: 42
Variables: 8
$ sample              <chr> "S01", "S02", "S03", "S04", "S05", "S06", "S07"...
$ samp_nbr            <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...
$ bull                <chr> "Galant", "Numero", "Speedy", "Borat", "Basudan...
$ all_high_fert       <chr> "NO", "NO", "YES", "YES", "YES", "NO", "YES", "...
$ kviga_high_fert     <chr> "NO", "YES", "YES", NA, "YES", "NO", "YES", "YE...
$ all_high_fert_val   <dbl> 39.1, 40.6, 66.7, 52.6, 57.4, 34.5, 56.2, 57.3,...
$ kviga_high_fert_val <dbl> 50.0, 81.0, 87.0, NA, 71.4, 51.8, 76.5, 81.3, 7...
$ in_mid_range        <chr> "NO", "NO", "NO", "NO", "NO", "NO", "NO", "NO",...


In [37]:
glimpse(data_df)

Observations: 1,110
Variables: 44
$ `Protein IDs`          <chr> "Q56J78|Q56J78_BOVIN;F1MMW8|F1MMW8_BOVIN;A0A...
$ `Majority protein IDs` <chr> "Q56J78|Q56J78_BOVIN;F1MMW8|F1MMW8_BOVIN;A0A...
$ S01                    <dbl> 20.58044, NA, 20.73793, 19.89583, 17.02213, ...
$ S02                    <dbl> NA, NA, 20.73843, NA, NA, 26.09065, 19.60695...
$ S03                    <dbl> NA, NA, 20.40941, NA, NA, 26.86506, 21.15781...
$ S04                    <dbl> NA, NA, 20.22782, 19.23612, NA, 24.70535, 20...
$ S05                    <dbl> NA, NA, 19.12194, NA, NA, 23.98497, 18.41724...
$ S06                    <dbl> 18.13740, NA, 20.99079, NA, NA, 26.16666, 21...
$ S07                    <dbl> 19.45939, NA, 22.39194, 21.47710, 18.81876, ...
$ S08                    <dbl> NA, NA, 19.69231, NA, NA, 26.49150, 20.74524...
$ S09                    <dbl> NA, NA, 19.61289, 18.76347, 19.49892, 25.873...
$ S10                    <dbl> 16.62100, NA, 20.20358, NA, 17.99362, 25.926...
$ S11             